In [ ]:
import os
import numpy as np
import pickle
import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# the pkl file dumped by `gen_repr.py`
rpath = '/data/ziyu/ood-cifar-inl-withclip.pkl'
reprs = pickle.load(open(rpath, 'rb'))
list(reprs)

In [ ]:
def normalize_feature(inl_train, ood, batch_dims=1):
    inl_train = inl_train.reshape([np.prod(inl_train.shape[:batch_dims]), -1])
    ood = ood.reshape([np.prod(ood.shape[:batch_dims]), -1])
    _mean = inl_train.mean(axis=0)[None]
    _sd = inl_train.std(axis=0)[None]
    return (ood - _mean) / _sd

def autocorr5(x, lags):
    '''
    adapted from https://stackoverflow.com/a/51168178/7509266
    Fixed the incorrect denominator: np.correlate(x,x,'full')[d-1+k] returns
        sum_{i=k}^{d-1} x[i]x[i-k]
    so we should divide it by (d-k) instead of d
    '''
    mean=x.mean()
    var=np.var(x)
    xp=x-mean
    ruler = len(x) - np.arange(len(x))
    corr=np.correlate(xp,xp,'full')[len(x)-1:]/var/ruler
    return corr[:(lags)]

def get_autocorr(fea, B=None, L=200, test_typ='bp', skip_corr=1):
    idcs = np.arange(fea.shape[0])
    if B is not None:
        np.random.shuffle(idcs)
    else:
        B = idcs.shape[0]
    corrs = []
    N = fea[0].shape[0]
    for j in tqdm.trange(B):
        ac_j = autocorr5(fea[idcs[j]], L)
        corrs.append(ac_j)
    corrs = np.array(corrs)[:,::skip_corr]
    if test_typ == 'ljb':
        ruler = (N - np.arange(1, L+1))[None, ::skip_corr].astype('f')
        stats = N * (N+2) * (corrs[:,1:]**2 / ruler[:,1:]).mean(axis=-1)  # normalized; *L would follow ChiSq[L]
    else:
        stats = N * (corrs[:,1:]**2).astype('f').mean(axis=-1)
    return stats, corrs

def plot_autocorrelations(corrs):
    plt.plot(np.arange(corrs.shape[1]), np.mean(corrs, axis=0))
    plt.fill_between(np.arange(corrs.shape[1]),
                     np.mean(corrs, axis=0)-np.std(corrs, axis=0), np.mean(corrs, axis=0)+np.std(corrs, axis=0), alpha=0.1)


def autocorr_plot(fea, B=None, L=200, test_typ='bp', skip_corr=1):
    stats, corrs = get_autocorr(fea, B, L, test_typ, skip_corr)
    plot_autocorrelations(corrs)
    return stats

In [ ]:
LW = reprs['inl_train'][2].shape[1]; LW

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

def get_roc(fea_inl, fea_ood):
    assert len(fea_inl.shape) == 1
    y = np.concatenate([np.ones_like(fea_inl), np.zeros_like(fea_ood)], axis=0)
    scores = np.concatenate([(fea_inl), (fea_ood)], axis=0)
    fpr, tpr, thrs = roc_curve(y, scores)
    auc = roc_auc_score(y, scores)
    return fpr, tpr, auc

def plot_roc(fea_inl, fea_ood):
    fpr, tpr, auc = get_roc(fea_inl, fea_ood)
    plt.plot(fpr, tpr)
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    plt.title('ROC curve, auc = {:.2f}'.format(auc))

def proc_scores(inl, oul):
    inl_mean = np.median(inl)
    return -np.abs(inl-inl_mean), -np.abs(oul-inl_mean)

In [ ]:
def logp_ms(TrLP, TLP, OOP, Bsz, Nsamples=10000, singleside=False):
    TrLP_normalized = TrLP.reshape((-1,LW*LW)).mean(axis=-1)
    TLP_normalized = TLP.reshape((-1,LW*LW)).mean(axis=-1)
    OLP_normalized = OOP.reshape((-1,LW*LW)).mean(axis=-1)
    
    def proc_logp(logps, bsz):
        N = logps.shape[0]
        ss = []
        for _ in range((Nsamples+N-1) * bsz // N):
            idcs = np.arange(N)
            np.random.shuffle(idcs)
            logps = logps[idcs]
            for k in range(0, N, bsz):
                ss.append(logps[k: k+bsz].mean())
        return np.array(ss)
    
    aucs = []
    for bsz in Bsz:
        inl_s = proc_logp(TLP_normalized, bsz)
        oul_s = proc_logp(OLP_normalized, bsz)
        if not singleside:
            _, _, auc = get_roc(*proc_scores(inl_s, oul_s))
        else:
            _, _, auc = get_roc(inl_s, oul_s)
        aucs.append(auc)

    return aucs

In [ ]:
def time_series_test(
    inl_train_fea, inl_test_fea, oul_fea, test_typ, Bsz=1, Nsamples=None,
    L=100, SK=1):
    """
    :param Bsz: batch size in a multi-sample test (as in the multi-sample typicality test, arXiv:1906.02994).
    :param SK: only include lags which are multiples of SK in the test
    :param L: the maximum lag to use
    """
    oul_fea = normalize_feature(inl_train_fea, oul_fea)
    inl_fea = normalize_feature(inl_train_fea, inl_test_fea)
    oul_stats, _ = get_autocorr(oul_fea, Nsamples, L, test_typ, SK)
    inl_stats, _ = get_autocorr(inl_fea, Nsamples, L, test_typ, SK)
    return get_roc(-inl_stats, -oul_stats)[2]

# The Proposed WN Test

In [ ]:
BSZ = [1]

for k in list(reprs):
    if k.startswith('od'):
        ktrain = 'inl_train'
        ktest = 'inl_test'
    elif k == 'inl_test':
        ktrain = ktest = 'inl_train'
        continue
    else:
        continue
    auc = time_series_test(
        reprs[ktrain][1][...], reprs[ktest][1][...], reprs[k][1][...], 'bp',
        Bsz=BSZ, L=400*3, SK=LW*3, normalize=True)
    print(k, auc)

# Baselines

## The Likelihood Tests

You can modify `BSZ` below to run the multi-sample typicality test.

### 2-Side

In [ ]:
BSZ = [1]

for k in list(reprs):
    if k.startswith('od'):
        ktrain = 'inl_train'
        ktest = 'inl_test'
    elif k == 'inl_test':
        ktrain = ktest = 'inl_train'
    else:
        continue
    aucs = logp_ms(reprs[ktrain][0], reprs[ktest][0], reprs[k][0], BSZ, singleside=False)
    print(k, aucs)

### Single-Side

In [ ]:
for k in list(reprs):
    if k.startswith('od'):
        ktrain = 'inl_train'
        ktest = 'inl_test'
    elif k == 'inl_test':
        ktrain = ktest = 'inl_train'
    else:
        continue
    aucs = logp_ms(reprs[ktrain][0], reprs[ktest][0], reprs[k][0], BSZ, singleside=True)
    print(k, aucs)

## The Compression Test

In [ ]:
import cv2, scipy

In [ ]:
# `discretized_mix_logistic_loss` returns log likelihood in nats. The log likelihood dumped by `gen_reprs.py` is a NHW tensor with each 
# dimension corresponding to the log joint pdf of the three subpixels.
# Thus in the following, we divide the input logp by 3, negate it to get nats-per-dimension, and subtract `BPD_PNG * ln(2) = NatsPD_PNG`. 
# The result is the negated LR test statistics, so the higher it is, the more likely the input is considered as outlier.

def compression_stats(logp, orig_ims):
    stats = []
    LW = orig_ims.shape[1]
    for k in tqdm.trange(orig_ims.shape[0]):
        rval, buf = cv2.imencode('.png', orig_ims[k], [int(cv2.IMWRITE_PNG_COMPRESSION), 9])
        assert rval, "PNG compression failed"
        bpd_png = 8 * buf.shape[0] / (LW*LW*3)
        stats.append(-logp[k].mean() / 3 - bpd_png * np.log(2))
    return stats

In [ ]:
inl_train_compression_stats = compression_stats(reprs['inl_train'][0], reprs['inl_train'][2])
inl_test_compression_stats = compression_stats(reprs['inl_test'][0], reprs['inl_test'][2])
print('inlier train vs test',
      get_roc(-np.array(inl_train_compression_stats), -np.array(inl_test_compression_stats))[2])

In [ ]:
for k in list(reprs):
    if k.startswith('od'):
        k_compression_stats = compression_stats(reprs[k][0], reprs[k][2])
        auc = get_roc(-np.array(inl_test_compression_stats), -np.array(k_compression_stats))[2]
        print(k, auc)

# Visualization and Diagnostics


## Distribution of ACFs (Fig. 4)

In [ ]:
if rpath.find('cifar-') != -1:
    oul_fea = normalize_feature(reprs['inl_train'][1][...], reprs['od_svhn'][1][...])
    inl_fea = normalize_feature(reprs['inl_train'][1][...], reprs['inl_test'][1][...])
    plt.figure(figsize=(7,2.5), facecolor='w')
    plt.subplot(121)
    plt.title('outlier ACF')
    oul_stats = autocorr_plot(oul_fea, L=500, B=5000, test_typ='bp', skip_corr=1, normalize=True)
    X = np.arange(500)
    plt.plot(X, 1/np.sqrt(1024*3)*np.ones((500,)), linestyle='--', color='gray')
    plt.plot(X, -1/np.sqrt(1024*3)*np.ones((500,)), linestyle='--', color='gray')
    plt.ylim(-0.05, 0.25)
    plt.subplot(122)
    plt.title('inlier ACF')
    inl_stats = autocorr_plot(inl_fea, L=500, B=5000, test_typ='bp', skip_corr=1, normalize=True)
    plt.ylim(-0.05, 0.25)
    plt.plot(X, 1/np.sqrt(1024*3)*np.ones((500,)), linestyle='--', color='gray')
    plt.plot(X, -1/np.sqrt(1024*3)*np.ones((500,)), linestyle='--', color='gray')

## Sensitivity to L (Fig. 5)

In [ ]:
if rpath.find('cifar-') != -1:
    
    # NOTE: we are using a small subset of data to speed up the process. There will be an error of ~0.03.
    # Fig.5 was generated with Nsamples=None
    Nsamples = 1000

    ac_results = {}

    for k in list(reprs):
        if k.startswith('od'):
            ktrain = 'inl_train'
            ktest = 'inl_test'
        elif k == 'inl_test':
            ktrain = ktest = 'inl_train'
        else:
            continue
        cr = []
        for L in [200, 400, 700]:  # max 1024=32x32
            auc = time_series_test(
                reprs[ktrain][1][...], reprs[ktest][1][...], reprs[k][1][...], 'bp',
                Nsamples=Nsamples, Bsz=BSZ, L=L*3, SK=LW*3, normalize=True)
            cr.append(auc)
        ac_results[k] = cr
        
    plt.figure(figsize=(10,2), facecolor='w')
    j = 0
    for k in ac_results:
        if k != 'od_celeba32' and k != 'od_imagenet' and k != 'od_svhn':
            continue
        j += 1
        plt.subplot(1, 3, j)
        plt.plot([200*3, 400*3, 700*3], ac_results[k], '-+')
        plt.xlabel('L')
        if j == 1:
            plt.ylabel('AUROC')
        dispname = {
            'od_celeba32': 'CelebA',
            'od_imagenet': 'TinyImageNet',
            'od_svhn': 'SVHN'
        }[k]
        plt.title(dispname)
        #plt.legend()

## Fig. 6

In [ ]:
def phist(inp, lbl): _ = plt.hist(inp, bins=100, alpha=0.4, density=True, label=lbl)

if 'od_synth-bus_sep' in reprs:
    plt.figure(figsize=(8,2))
    plt.subplot(131)
    phist(np.log(2) * -reprs['inl_test'][0].mean(axis=(1,2))/3, 'Inlier')
    phist(np.log(2) * -reprs['od_synth-bus_sep'][0].mean(axis=(1,2))/3, 'Outlier')
    plt.title('Model BPD')
    plt.legend()
    plt.subplot(132)
    phist(np.log(2) * (-np.array(inl_test_compression_stats)-reprs['inl_test'][0].mean(axis=(1,2))/3), 'Inlier')
    phist(np.log(2) * (-np.array(k_compression_stats)-reprs['od_synth-bus_sep'][0].mean(axis=(1,2))/3), 'Outlier')
    #plt.legend()
    plt.title('Generic BPD')
    plt.subplot(133)
    phist(np.log(2) * (-np.array(inl_test_compression_stats)), 'Inlier')
    phist(np.log(2) * (-np.array(k_compression_stats)), 'Outlier')
    #plt.legend()
    plt.title('Test Stats')